## 1. [[Boj] 16236. 아기상어](https://www.acmicpc.net/problem/16236)

- 크기가 큰 물고기는 지나갈 수 없음, 먹을 수 없음
- 크기가 같은 물고기는 지나갈 수 있음, 먹을 수 없음
- 크기가 작은 물고기는 지나갈 수 있음, 먹을 수 있음


- 가장 가까운 물고기부터, 
- 가까운 물고기가 많다면 가장 위, 왼쪽에 있는 물고기부터
--> 갈 수 있는 모든 상황에 대해 우선순위를 부여해야 함 (deque X, heapq O)

In [112]:
# 우선순위 큐 사용
# (x,y) x가 작은 순대로, y가 작은 순대로 실행
import heapq

n = int(input())
arr = []
for i in range(n):
    arr.append(list(map(int,input().split())))
    for j in range(n):
        if arr[i][j] == 9:
            x,y = (i,j)

dx = [-1,+1,0,0] # 위, 아래
dy = [0,0,-1,+1] # 왼쪽, 오른쪽
ans = 0 # 최종 답
t = 0 # 시간
w = 2 #상어 무게
visited = [[False]*n for _ in range(n)]

q = [[(x,y)]]
visited[x][y] = True
arr[x][y] = 0

cnt = 0 # 물고기 잡아먹은 횟수
while q:
    t += 1
    tmp = [] # 최소 힙
    p = q.pop() # p = tmp
    while p:
        x, y = heapq.heappop(p)
        for i in range(4):
            nx = x + dx[i]
            ny = y + dy[i]
            if nx in range(n) and ny in range(n) \
            and not visited[nx][ny] and arr[nx][ny] in range(w+1): # 0~w까지 지나갈 수 있음
                # 경로 담기 (경로를 모두 담은 후에, 위쪽, 왼쪽 순으로 뽑아서 먹을 수 있는지 확인해야 함)
                # 발견하자마자 먹으면 가장 위쪽, 왼쪽에 해당하는 경로가 아닐 수 있기 때문에
                visited[nx][ny] = True
                heapq.heappush(tmp, (nx,ny))
    
    if not tmp: # tmp에 아무것도 없으면 반복문 종료
        break
    
    # 물고기 잡아먹기 (무게가 1~w-1인 경우)
    tmp_copy = [i[:] for i in tmp]
    while tmp:
        x,y = heapq.heappop(tmp)
        if arr[x][y] in range(1,w):
            arr[x][y] = 0
            cnt += 1 # 물고기 잡아먹은 횟수 + 1
            ans = t
            # 한마리 잡아먹었으면 그 지점부터 다시 탐색 시작
            q = ([[(x,y)]])
            visited = [[False]*n for _ in range(n)]
            visited[x][y] = True
            break
    else: # 물고기 안잡아먹었으면(=break문에 안걸리면) q append하기
        q.append(tmp_copy)
        
    if cnt == w: # 상어 무게 확인
        cnt = 0 # 초기화
        w += 1
print(ans)

60


## 2. [[Boj] 1967. 트리](https://www.acmicpc.net/problem/1967)

In [27]:
from collections import deque

n = int(input()) # 노드의 개수
graph = [[] for _ in range(n+1)]
for _ in range(n-1):
    a, b, c = map(int,input().split())
    graph[a].append((b,c))
    graph[b].append((a,c))
    
ans = 0
for start in range(1,n+1):
    q = deque([])
    dist = [0] * (n+1)
    visited = [False] * (n+1)
    q.append((0,start))
    visited[start] = True

    while q:
        now_d, now_n = q.popleft()
        for next_n, next_d in graph[now_n]:
            if not visited[next_n]:
                cost = now_d + next_d
                visited[next_n] = True
                if cost > dist[next_n]:
                    dist[next_n] = cost
                    q.append((cost,next_n))
    ans = max(ans, max(dist))
print(ans)

12
1 2 3
1 3 2
2 4 5
3 5 11
3 6 9
4 7 1
4 8 7
5 9 15
5 10 4
6 11 6
6 12 10
45


## 3. [[Boj] 1726. 로봇](https://www.acmicpc.net/problem/1726)

In [264]:
from collections import deque

m, n = map(int,input().split()) # 행, 열
arr = [list(map(int,input().split())) for _ in range(m)] # 0 - 지나는 점, 1 - 장애물
x1,y1,d1 = map(int,input().split())
x2,y2,d2 = map(int,input().split())

# 동(1), 남(2), 서(3), 북(0)
loc = {1:[0,+1], 2:[+1,0], 3:[0,-1], 0:[-1,0]}
dic = {1:1, 2:3, 3:2, 4:0}
d1 = dic[d1]
d2 = dic[d2]

visited = [[[False]*4 for _ in range(n)] for _ in range(m)]
q = deque([(x1-1,y1-1, d1,0)])
visited[x1-1][y1-1][d1] = True

while q:
    x,y,d,cnt = q.popleft()
    if (x,y,d) == (x2-1,y2-1,d2):
        print(cnt)
        break
    # 1, 2, 3칸 전진
    for i in range(1,4):
        nx = x + loc[d][0] * i
        ny = y + loc[d][1] * i
        if nx in range(m) and ny in range(n) and arr[nx][ny] == 0:# and (nx,ny,d,cnt+1) not in q:
            if not visited[nx][ny][d]:
                visited[nx][ny][d] = True # 방문 표시
                q.append((nx,ny,d,cnt+1))
        
        else: # 1번째가 막혀있다면 2, 3번째도 당연히 못지나감
            break
            
    # 방향 바꾸기
    for nd in [(d+1)%4, (d-1)%4]: # 오른쪽, 왼쪽
        if not visited[x][y][nd]:
            q.append((x,y,nd, cnt+1))
            visited[x][y][nd] = True

4 2
0 0
0 0
1 0
0 0
1 1 3
4 1 3
7
